In [8]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
from astropy.table import Table

In [5]:
def combine_deep(csfq, ssfq,mass_bin, z, mode='count',path = 'CUT_deep_catalogs/calibration_deep/'):
    
    def massive_counts(cat_name,z,csfq):
        cat = Table.read('CUT_deep_catalogs/massive_counts_inside/'+cat_name+'_0.09_11.15.fits')
        cat = cat[cat['zphot']==z]
        if len(cat)==0:
            return 0
        
        if csfq=='csf':
            return cat['count_csf'][0]
        elif csfq=='cq':
            return cat['count_cq'][0]
        else:
            return cat['count'][0]
            
    radial_COSMOS = np.load(path+mode+'COSMOS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'.npy')
    radial_DEEP = np.load(path+mode+'DEEP_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'.npy')
    radial_SXDS = np.load(path+mode+'SXDS_uddd_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'.npy')
    radial_ELAIS = np.load(path+mode+'ELAIS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'.npy')
    radial_XMM = np.load(path+mode+'XMM-LSS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'.npy')
    
    radial_COSMOS_err = np.load(path+mode+'COSMOS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'_err.npy')
    radial_DEEP_err = np.load(path+mode+'DEEP_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'_err.npy')
    radial_SXDS_err = np.load(path+mode+'SXDS_uddd_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'_err.npy')
    radial_ELAIS_err = np.load(path+mode+'ELAIS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'_err.npy')
    radial_XMM_err = np.load(path+mode+'XMM-LSS_deep_'+mass_bin + '_' + csfq + '_' + ssfq+'_'+str(z)+'_err.npy')
    
    count_COSMOS = massive_counts('COSMOS_deep',z,csfq)
    count_DEEP = massive_counts('DEEP_deep',z,csfq)
    count_SXDS = massive_counts('SXDS_uddd',z,csfq)
    count_ELAIS = massive_counts('ELAIS_deep',z,csfq)
    count_XMM = massive_counts('XMM-LSS_deep',z,csfq)
    count_total = sum([count_COSMOS,count_DEEP,count_SXDS,count_ELAIS,count_XMM])
    print(count_total)

    radial_total = (radial_COSMOS*count_COSMOS+radial_XMM*count_XMM+radial_ELAIS*count_ELAIS+radial_SXDS*count_SXDS+radial_DEEP*count_DEEP)/count_total
    radial_total_err = np.sqrt(radial_COSMOS_err**2*count_COSMOS**2+radial_XMM_err**2*count_XMM**2+radial_ELAIS_err**2*count_ELAIS**2+radial_SXDS_err**2*count_SXDS**2+radial_DEEP_err**2*count_DEEP**2)/count_total
                               
    return radial_total, radial_total_err

In [9]:
def red_frac_err(q, sf, q_err, sf_err):
    return np.sqrt( (sf**2*q_err**2 + q**2*sf_err**2) / ((q+sf)**4) )

path = 'CUT_deep_catalogs/split_sat_mass/'
radial_3_sf_l = combine_deep('all','ssf','9.0',0.3,path=path)
radial_3_sf_m = combine_deep('all','ssf','9.5',0.3,path=path)
radial_3_sf_h = combine_deep('all','ssf','10.2',0.3,path=path)
radial_3_q_l = combine_deep('all','sq','9.0',0.3,path=path)
radial_3_q_m = combine_deep('all','sq','9.5',0.3,path=path)
radial_3_q_h = combine_deep('all','sq','10.2',0.3,path=path)

radial_6_sf_l = combine_deep('all','ssf','9.0',0.6,path=path)
radial_6_sf_m = combine_deep('all','ssf','9.5',0.6,path=path)
radial_6_sf_h = combine_deep('all','ssf','10.2',0.6,path=path)
radial_6_q_l = combine_deep('all','sq','9.0',0.6,path=path)
radial_6_q_m = combine_deep('all','sq','9.5',0.6,path=path)
radial_6_q_h = combine_deep('all','sq','10.2',0.6,path=path)

radial_3_l = combine_results([radial_3_sf_l[0],radial_3_q_l[0]],[radial_3_sf_l[1],radial_3_q_l[1]])
radial_6_l = combine_results([radial_6_sf_l[0],radial_6_q_l[0]],[radial_6_sf_l[1],radial_6_q_l[1]])
radial_3_m = combine_results([radial_3_sf_m[0],radial_3_q_m[0]],[radial_3_sf_m[1],radial_3_q_m[1]])
radial_6_m = combine_results([radial_6_sf_m[0],radial_6_q_m[0]],[radial_6_sf_m[1],radial_6_q_m[1]])
radial_3_h = combine_results([radial_3_sf_h[0],radial_3_q_h[0]],[radial_3_sf_h[1],radial_3_q_h[1]])
radial_6_h = combine_results([radial_6_sf_h[0],radial_6_q_h[0]],[radial_6_sf_h[1],radial_6_q_h[1]])

bin_edges = np.load('bin_edges.npy')

redfrac_err_3_l = red_frac_err(radial_3_q_l[0],radial_3_sf_l[0],radial_3_q_l[1],radial_3_sf_l[1])
redfrac_err_6_l = red_frac_err(radial_6_q_l[0],radial_6_sf_l[0],radial_6_q_l[1],radial_6_sf_l[1])
redfrac_err_3_m = red_frac_err(radial_3_q_m[0],radial_3_sf_m[0],radial_3_q_m[1],radial_3_sf_m[1])
redfrac_err_6_m = red_frac_err(radial_6_q_m[0],radial_6_sf_m[0],radial_6_q_m[1],radial_6_sf_m[1])
redfrac_err_3_h = red_frac_err(radial_3_q_h[0],radial_3_sf_h[0],radial_3_q_h[1],radial_3_sf_h[1])
redfrac_err_6_h = red_frac_err(radial_6_q_h[0],radial_6_sf_h[0],radial_6_q_h[1],radial_6_sf_h[1])

print(redfrac_err_3_m)

plt.rc('font', family='serif'), plt.rc('xtick', labelsize=15), plt.rc('ytick', labelsize=15)
fig, axs = plt.subplots(2, 3, sharex=True, sharey='row', figsize=(16, 8.5),gridspec_kw={'height_ratios':[3,1]})
# Remove horizontal space between axes
fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)
bin_edges = np.load('bin_edges.npy')

#====================Radial Distribution ===================

# Total curves
axs[0][0].errorbar(bin_edges[1:], radial_3_l[0], yerr=None, fmt='k',linewidth=2)
axs[0][0].fill_between(bin_edges[1:],radial_3_l[0]+radial_3_l[1], 
                       radial_3_l[0]-radial_3_l[1], alpha=0.2, color='k')

axs[0][1].errorbar(bin_edges[1:], radial_3_m[0], yerr=None, fmt='k',linewidth=2)
axs[0][1].fill_between(bin_edges[1:],radial_3_m[0]+radial_3_m[1], 
                       radial_3_m[0]-radial_3_m[1], alpha=0.2, color='k')

axs[0][2].errorbar(bin_edges[1:], radial_3_h[0], yerr=None,fmt='k',label='total')
axs[0][2].fill_between(bin_edges[1:],radial_3_h[0]+radial_3_h[1], 
                       radial_3_h[0]-radial_3_h[1], alpha=0.2, color='k')

axs[0][0].errorbar(bin_edges[1:], radial_6_l[0], yerr=None, fmt='--k',linewidth=2)
axs[0][0].fill_between(bin_edges[1:],radial_6_l[0]+radial_6_l[1], 
                       radial_6_l[0]-radial_6_l[1], alpha=0.2, color='k')

axs[0][1].errorbar(bin_edges[1:], radial_6_m[0], yerr=None, fmt='--k',linewidth=2)
axs[0][1].fill_between(bin_edges[1:],radial_6_m[0]+radial_6_m[1], 
                       radial_6_m[0]-radial_6_m[1], alpha=0.2, color='k')

axs[0][2].errorbar(bin_edges[1:], radial_6_h[0], yerr=None,fmt='--k')
axs[0][2].fill_between(bin_edges[1:],radial_6_h[0]+radial_6_h[1], 
                       radial_6_h[0]-radial_6_h[1], alpha=0.2, color='k')

# Star Forming curves
axs[0][0].errorbar(bin_edges[1:], radial_3_sf_l[0], yerr=None, fmt='b',label='z=0.3',linewidth=2)
axs[0][0].errorbar(bin_edges[1:], radial_6_sf_l[0], yerr=None, fmt='--b',label='z=0.6',linewidth=2)

axs[0][0].fill_between(bin_edges[1:],radial_3_sf_l[0]+radial_3_sf_l[1], 
                       radial_3_sf_l[0]-radial_3_sf_l[1], alpha=0.2, color='b')
axs[0][0].fill_between(bin_edges[1:],radial_6_sf_l[0]+radial_6_sf_l[1], 
                       radial_6_sf_l[0]-radial_6_sf_l[1], alpha=0.2, color='b')


axs[0][0].set_title(r'9.0<$\log(M/M_\odot)$<9.5',fontsize=14)
axs[0][0].set_xlabel('Projected Radius [kpc]', fontsize=14)
axs[0][0].set_ylabel(r'N kpc$^{-2}$ Central$^{-1}$ dr',fontsize=14)

axs[0][1].errorbar(bin_edges[1:], radial_3_sf_m[0], yerr=None, fmt='b',linewidth=2)
axs[0][1].errorbar(bin_edges[1:], radial_6_sf_m[0], yerr=None, fmt='--b',linewidth=2)

axs[0][1].fill_between(bin_edges[1:],radial_3_sf_m[0]+radial_3_sf_m[1], 
                       radial_3_sf_m[0]-radial_3_sf_m[1], alpha=0.2, color='b')
axs[0][1].fill_between(bin_edges[1:],radial_6_sf_m[0]+radial_6_sf_m[1], 
                       radial_6_sf_m[0]-radial_6_sf_m[1], alpha=0.2, color='b')

axs[0][1].set_xlabel('Projected Radius [kpc]', fontsize=14)
axs[0][1].set_title(r'9.50<$\log(M/M_\odot)$<10.2',fontsize=14)

axs[0][2].errorbar(bin_edges[1:], radial_3_sf_h[0], yerr=None,fmt='b',label='sf')
axs[0][2].errorbar(bin_edges[1:], radial_6_sf_h[0], yerr=None,fmt='--b')
axs[0][2].fill_between(bin_edges[1:],radial_3_sf_h[0]+radial_3_sf_h[1], 
                       radial_3_sf_h[0]-radial_3_sf_h[1], alpha=0.2, color='b')
axs[0][2].fill_between(bin_edges[1:],radial_6_sf_h[0]+radial_6_sf_h[1], 
                       radial_6_sf_h[0]-radial_6_sf_h[1], alpha=0.2, color='b')
axs[0][2].set_title(r'10.2<$\log(M/M_\odot)$',fontsize=14)
axs[0][2].legend(fontsize=14)


# Quiescent Curves
axs[0][0].errorbar(bin_edges[1:], radial_3_q_l[0], yerr=None,fmt='r')
axs[0][0].errorbar(bin_edges[1:], radial_6_q_l[0], yerr=None,fmt='--r')
axs[0][0].fill_between(bin_edges[1:],radial_3_q_l[0]+radial_3_q_l[1], 
                       radial_3_q_l[0]-radial_3_q_l[1], alpha=0.2, color='r')
axs[0][0].fill_between(bin_edges[1:],radial_6_q_l[0]+radial_6_q_l[1], 
                       radial_6_q_l[0]-radial_6_q_l[1], alpha=0.2, color='r')
axs[0][0].legend(fontsize=14)

axs[0][1].errorbar(bin_edges[1:], radial_3_q_m[0], yerr=None,fmt='r',linewidth=2)
axs[0][1].errorbar(bin_edges[1:], radial_6_q_m[0], yerr=None,fmt='--r',linewidth=2)
axs[0][1].fill_between(bin_edges[1:],radial_3_q_m[0]+radial_3_q_m[1], 
                       radial_3_q_m[0]-radial_3_q_m[1], alpha=0.2, color='r')
axs[0][1].fill_between(bin_edges[1:],radial_6_q_m[0]+radial_6_q_m[1], 
                       radial_6_q_m[0]-radial_6_q_m[1], alpha=0.2, color='r')

axs[0][2].errorbar(bin_edges[1:], radial_3_q_h[0], yerr=None,fmt='r',label='q',linewidth=2)
axs[0][2].errorbar(bin_edges[1:], radial_6_q_h[0], yerr=None,fmt='--r',linewidth=2)
axs[0][2].fill_between(bin_edges[1:],radial_3_q_h[0]+radial_3_q_h[1], 
                       radial_3_q_h[0]-radial_3_q_h[1], alpha=0.2, color='r')
axs[0][2].fill_between(bin_edges[1:],radial_6_q_h[0]+radial_6_q_h[1], 
                       radial_6_q_h[0]-radial_6_q_h[1], alpha=0.2, color='r')
axs[0][2].legend(fontsize=14)

axs[0][0].set_yscale('log')
axs[0][0].set_xscale('log')
axs[0][0].set_xticks([20, 100, 400])
axs[0][0].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

#================Red Fraction========================
axs[1][0].errorbar(bin_edges[1:], radial_3_q_l[0]/(radial_3_sf_l[0]+radial_3_q_l[0]),fmt='k',yerr=redfrac_err_3_l,label='z=0.3')
axs[1][0].errorbar(bin_edges[1:], radial_6_q_l[0]/(radial_6_sf_l[0]+radial_6_q_l[0]),fmt='--k',yerr=redfrac_err_6_l,label='z=0.6')
axs[1][0].set_ylabel('Red Fraction',fontsize=14)
axs[1][0].legend(fontsize=14)
axs[1][0].set_ylim([0,1])

axs[1][1].errorbar(bin_edges[1:], radial_3_q_m[0]/(radial_3_sf_m[0]+radial_3_q_m[0]),fmt='k',yerr=redfrac_err_3_m)
axs[1][1].errorbar(bin_edges[1:], radial_6_q_m[0]/(radial_6_sf_m[0]+radial_6_q_m[0]),fmt='--k',yerr=redfrac_err_6_m)

axs[1][2].errorbar(bin_edges[1:], radial_3_q_h[0]/(radial_3_sf_h[0]+radial_3_q_h[0]),fmt='k',yerr=redfrac_err_3_h)
axs[1][2].errorbar(bin_edges[1:], radial_6_q_h[0]/(radial_6_sf_h[0]+radial_6_q_h[0]),fmt='--k',yerr=redfrac_err_6_h)

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
plt.grid(False)
plt.xlabel('Projected Radius [kpc]', fontsize=18)
plt.savefig('splitting_redfraction.png')
plt.show()

0
0
0
0
0


/home/lejay/.local/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in divide
/home/lejay/.local/lib/python2.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in divide


0
1450.0
1450.0
1450.0


IOError: [Errno 2] No such file or directory: 'CUT_deep_catalogs/split_sat_mass/countELAIS_deep_9.0_all_sq_0.6.npy'